# Generating Example Data

We use data from the Virtual Cell Challenge for model training, inference, and finetuning.

In [ ]:
'''
Download the dataset

(taken from Colab Notebook by Adduri et al.
https://colab.research.google.com/drive/1QKOtYP7bMpdgDJEipDxaJqOchv7oQ-_l#scrollTo=h0aSjKX7Rtyw)
'''

import requests
from tqdm.auto import tqdm  # picks the best bar for the environment
from zipfile import ZipFile
from tqdm.auto import tqdm
import os

# Download the Replogle-Nadig training dataset.
url = "https://storage.googleapis.com/vcc_data_prod/datasets/state/competition_support_set.zip"
output_path = "competition_support_set.zip"

# stream the download so we can track progress
response = requests.get(url, stream=True)
total = int(response.headers.get("content-length", 0))

with open(output_path, "wb") as f, tqdm(
    total=total, unit='B', unit_scale=True, desc="Downloading"
) as bar:
    for chunk in response.iter_content(chunk_size=8192):
        if not chunk:
            break
        f.write(chunk)
        bar.update(len(chunk))

out_dir  = "competition_support_set"
os.makedirs(out_dir, exist_ok=True)
with ZipFile(output_path, 'r') as z:
    for member in tqdm(z.infolist(), desc="Unzipping", unit="file"):
        z.extract(member, out_dir)

# State embeddings

We can generate embeddings using the STATE Embedding model in the helical package.

In [3]:
from helical.models.state import stateEmbeddingsModel
from helical.models.state import stateConfig

state_config = stateConfig()
state_embed = stateEmbeddingsModel(configurer=state_config)

Fetching 3 files: 100%|██████████| 3/3 [00:00<00:00, 9383.23it/s]
INFO:helical.models.state.state_embeddings:Using model checkpoint: /home/rasched/.cache/helical/models/state/embed_files/se600m_epoch16.ckpt


In [5]:
import scanpy as sc

adata = sc.read_h5ad("competition_support_set/competition_val_template.h5ad")
adata = adata[:2].copy()

processed_data = state_embed.process_data(adata=adata)
embeddings = state_embed.get_embeddings(processed_data)

print(embeddings.shape)

INFO:helical.models.state.model_dir._embed_utils.inference:Auto-detected gene column: var.index (overlap: 17773/19790 protein embeddings, 98.3% of genes)
INFO:/home/rasched/final_helical_with_state/helical/helical/models/state/model_dir/_embed_utils/data/loader.py:17773 genes mapped to embedding file (out of 18080)


!!! 17773 genes mapped to embedding file (out of 18080)


INFO:helical.models.state.state_embeddings:Successfully processed the data for State Embeddings.
Encoding: 100%|██████████| 1/1 [00:00<00:00,  1.70it/s]
INFO:helical.models.state.state_embeddings:Finished getting embeddings.


(4116,)


# Training the Model

The example data should generate a directory. Before training edit the `competition_support_set/starter.toml` file to point to the correct path on your machine. Here you can also change the train/test split but feel free to leave this at default.

In [1]:
# train the model on the training data
from helical.models.state import stateTransitionTrainModel
from helical.models.state.train_configs import trainingConfig

train_config = trainingConfig(
    output_dir="competition",
    name="first_run",
    toml_config_path="competition_support_set/starter.toml",
    checkpoint_name="final.ckpt",
    max_steps=40000,
    max_epochs=1,
    ckpt_every_n_steps=20000,
    num_workers=4,
    batch_col="batch_var",
    pert_col="target_gene",
    cell_type_key="cell_type",
    control_pert="non-targeting",
    perturbation_features_file="competition_support_set/ESM2_pert_features.pt"
    )

state_train = stateTransitionTrainModel(configurer=train_config)
state_train.train() 
state_train.predict() 

INFO:datasets:PyTorch version 2.6.0 available.
INFO:datasets:Polars version 1.33.0 available.
INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42
INFO:cell_load.config:Configuration validation passed
INFO:cell_load.data_modules.perturbation_dataloader:Initializing DataModule: batch_size=16, workers=4, random_seed=42


/home/rasched/final_helical_with_state/helical/helical/models/state/competition_support_set/{competition_train,k562_gwps,rpe1,jurkat,k562,hepg2}.h5


INFO:cell_load.data_modules.perturbation_dataloader:Set 2 missing perturbations to zero vectors.
INFO:cell_load.data_modules.perturbation_dataloader:Loaded custom perturbation featurizations for 19792 perturbations.
INFO:cell_load.data_modules.perturbation_dataloader:Processing dataset replogle_h1:
INFO:cell_load.data_modules.perturbation_dataloader:  - Training dataset: True
INFO:cell_load.data_modules.perturbation_dataloader:  - Zeroshot cell types: ['hepg2']
INFO:cell_load.data_modules.perturbation_dataloader:  - Fewshot cell types: []
Processing replogle_h1: 100%|██████████| 6/6 [00:00<00:00, 30.05it/s]
INFO:cell_load.data_modules.perturbation_dataloader:

INFO:cell_load.data_modules.perturbation_dataloader:Done! Train / Val / Test splits: 5 / 0 / 1


Processed competition_train: 221273 train, 0 val, 0 test
Processed k562_gwps: 111605 train, 0 val, 0 test
Processed rpe1: 22317 train, 0 val, 0 test
Processed jurkat: 21412 train, 0 val, 0 test
Processed k562: 18465 train, 0 val, 0 test
Processed hepg2: 0 train, 0 val, 9386 test
Model created. Estimated params size: 0.61 GB and 650505936 parameters


INFO:helical.models.state.state_train:Loggers and callbacks set up.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:helical.models.state.state_train:Starting trainer fit.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

INFO:cell_load.data_modules.samplers:Total # cells 395072. Cell set size mean / std before resampling: 117.72 / 27.93.
INFO:cell_load.data_modules.samplers:Creating meta-batches with cell_sentence_len=128...
INFO:cell_load.data_modules.samplers:Of all batches, 2831 were full and 525 were partial.
INFO:cell_load.data_modules.samplers:Sampler created with 210 batches in 0.47 seconds.
INFO:cell_load.data_modules.samplers:Of all batches, 2831 were full and 525 were partial.


Epoch 0: 100%|██████████| 210/210 [01:41<00:00,  2.06it/s, v_num=0]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 210/210 [01:41<00:00,  2.06it/s, v_num=0]
Training completed, saving final checkpoint...


INFO:cell_load.data_modules.samplers:Creating perturbation batch sampler with metadata caching (using codes)...
INFO:cell_load.data_modules.samplers:Total # cells 9386. Cell set size mean / std before resampling: 78.22 / 47.32.
INFO:cell_load.data_modules.samplers:Creating meta-batches with cell_sentence_len=128...
INFO:cell_load.data_modules.samplers:Of all batches, 120 were full and 0 were partial.
INFO:cell_load.data_modules.samplers:Sampler created with 120 batches in 0.01 seconds.
INFO:helical.models.state.state_train:Loading model from competition/first_run/final.ckpt
INFO:helical.models.state.state_train:Model loaded successfully.
INFO:helical.models.state.state_train:Generating predictions on test set using manual loop...
Predicting: 100%|██████████| 120/120 [00:13<00:00,  9.21batch/s]
INFO:helical.models.state.state_train:Creating anndatas from predictions from manual loop...
... storing 'target_gene' as categorical
... storing 'cell_type' as categorical
... storing 'batch_var

Once the model is trained we can perform inference on a new dataset using:

In [ ]:
from helical.models.state import stateTransitionModel

state_config = stateConfig(
    output = "competition/prediction.h5ad",
    model_dir = "competition/first_run",
    model_config = "model_dir/config.yaml",
    pert_col = "target_gene",
)

adata = sc.read_h5ad("competition_support_set/competition_val_template.h5ad")

state_transition = stateTransitionModel(configurer=state_config)
adata = state_transition.process_data(adata)
embeds = state_transition.get_embeddings(adata)

Using checkpoint: competition/first_run/final.ckpt
Model device: cpu
Model cell_set_len (max sequence length): 128
Model uses batch encoder: False
Model output space: all
Grouping by cell type column: cell_type
Using adata.X as input features
Cells: total=98927, control=38176, non-control=60751
Running virtual experiment (homogeneous per-perturbation forward passes; controls included)...


Group H1: 100%|██████████| 51/51 [00:28<00:00,  1.77it/s, Pert: non-targeting           ]



=== Inference complete ===
Input cells:         98927
Controls simulated:  38176
Treated simulated:   60751
Wrote predictions to adata.X
Saved:               competition/prediction.h5ad


# Creating a Virtual Cell Challenge Submission

To create a submission for the Virtual Cell Challenge we pass our previous prediction file into `helical.models.state.vcc_eval` which uses the `cell-eval` package. This will generate a `.vcc` file that can be uploaded to the public leaderboard.

In [7]:
# evaluate the model - underlying function uses cell-eval package 
# (https://github.com/ArcInstitute/cell-eval)
from helical.models.state import vcc_eval

# default configs for competition dataset
EXPECTED_GENE_DIM = 18080
MAX_CELL_DIM = 100000
DEFAULT_PERT_COL = "target_gene"
DEFAULT_CTRL = "non-targeting"
DEFAULT_COUNTS_COL = "n_cells"
DEFAULT_CELLTYPE_COL = "celltype"
DEFAULT_NTC_NAME = "non-targeting"

configs = {
    # path to the prediction file
    "input": "competition/prediction.h5ad",
    # path to the gene names file
    "genes": "competition_support_set/gene_names.csv",
    # path to the output file - if None will be created with default naming
    "output": None,
    "pert_col": DEFAULT_PERT_COL,
    "celltype_col": None,
    "ntc_name": DEFAULT_NTC_NAME,
    "output_pert_col": DEFAULT_PERT_COL,
    "output_celltype_col": DEFAULT_CELLTYPE_COL,
    "encoding": 32,
    "allow_discrete": False,
    "expected_gene_dim": EXPECTED_GENE_DIM,
    "max_cell_dim": MAX_CELL_DIM,
}

# this creates a submission file in the output directory which can be uploaded to the challenge leaderboard
vcc_eval(configs)

INFO:helical.models.state.model_dir._vcc_eval.vcc_eval:Reading input anndata
INFO:helical.models.state.model_dir._vcc_eval.vcc_eval:Reading gene list
INFO:helical.models.state.model_dir._vcc_eval.vcc_eval:Preparing anndata
INFO:helical.models.state.model_dir._vcc_eval.vcc_eval:Using 32-bit float encoding
INFO:helical.models.state.model_dir._vcc_eval.vcc_eval:Setting data to sparse if not already
INFO:helical.models.state.model_dir._vcc_eval.vcc_eval:Simplifying obs dataframe
INFO:helical.models.state.model_dir._vcc_eval.vcc_eval:Simplifying var dataframe
INFO:helical.models.state.model_dir._vcc_eval.vcc_eval:Creating final minimal AnnData object
INFO:helical.models.state.model_dir._vcc_eval.vcc_eval:Applying normlog transformation if required
INFO:helical.models.state.model_dir._vcc_eval._evaluator:Input is found to be log-normalized already - skipping transformation.
INFO:helical.models.state.model_dir._vcc_eval.vcc_eval:Writing h5ad output to /tmp/tmpv0elic7d/pred.h5ad
INFO:helical.m

# Finetuning Example

We can further finetune the model. This can start with a model directory containing a `.ckpt` file. If the model directory does not exists, one will be created and so will a new model instance. The finetuning will produce model weights and head weights that will be saved and training will resume from these `.pt` files in the directory (if present).

In [ ]:
from helical.models.state import stateFineTuningModel

# Load the desired dataset
adata = sc.read_h5ad("competition_support_set/competition_val_template.h5ad")

# Get the desired label class
cell_types = list(adata.obs.cell_type)
label_set = set(cell_types)

# Create the fine-tuning model (no need to specify var_dims location)
config = stateConfig(
    batch_size=8,
    model_dir="competition/first_run",
    model_config="model_dir/config.yaml",
    freeze_backbone=True
)

model = stateFineTuningModel(
    configurer=config, 
    fine_tuning_head="classification", 
    output_size=len(label_set),
)

# Process the data for training
data = model.process_data(adata)

# Create a dictionary mapping the classes to unique integers for training
class_id_dict = dict(zip(label_set, [i for i in range(len(label_set))]))

for i in range(len(cell_types)):
    cell_types[i] = class_id_dict[cell_types[i]]

# Fine-tune
model.train(train_input_data=data, train_labels=cell_types)

INFO:helical.models.state.state_finetune:Loading existing config.yaml from: configs/config.yaml
INFO:helical.models.state.state_finetune:Loading pre-trained model from: competition/first_run/final.ckpt
INFO:helical.models.state.state_finetune:Full model fine-tuning - both backbone and head will be trained
INFO:helical.models.state.state_finetune:Processing data for state model fine-tuning.
INFO:helical.models.state.state_finetune:Loaded perturbation mapping with 19792 perturbations
INFO:helical.models.state.state_finetune:Successfully processed the data for state model fine-tuning.
INFO:helical.models.state.state_finetune:Optimizer set up for full model fine-tuning
INFO:helical.models.state.state_finetune:Starting Fine-Tuning
Fine-Tuning: epoch 1/1: 100%|██████████| 12366/12366 [00:14<00:00, 861.43it/s, loss=0]
INFO:helical.models.state.state_finetune:Fine-Tuning Complete. Epochs: 1
